# Set up environment

In [ ]:
# Uncomment below section and run in case of re-connecting Colab

# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
# !pip install transformers
# !pip install git+https://github.com/ssut/py-hanspell.git

# from google.colab import drive
# drive.mount('/content/drive')

# %cd drive/MyDrive/MBTI
# !pwd

In [2]:
import torch
import pandas as pd
pd.set_option('display.width', 180)

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Prepare dataset & dataloader

In [5]:
# Setup
from dataloader import MBTIDataset

env_dict = {
    # ==== Arguments of dataset =====
    'train_path'        : './data/example_train.csv',
    'question_path'     : './data/question_filtered.csv',
    'target'            : 'E',
    'pretrained_url'    : "klue/bert-base",
    'padding_per_batch' : True,
    # ==== Arguments of dataloader =====
    'batch_size'        : 64,
    'shuffle'           : True

}

In [7]:
# Dataset
train_dataset = MBTIDataset(
    data_path           = env_dict['train_path'],
    question_path       = env_dict['question_path'],
    target_mbti         = env_dict['target'],
    pretrained_url      = env_dict['pretrained_url'],
    padding_per_batch   = env_dict['padding_per_batch'],
    is_train            = True
)

print(len(train_dataset))
print(train_dataset.data.head())

96
   Data_ID  User_ID  Gender       Age  I/E  Q_number                                             Answer                                        QandA
0        1        1       1 -0.994778    0         1  <아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...  [input_ids, token_type_ids, attention_mask]
1        2        1       1 -0.994778    0         2  <중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하며...  [input_ids, token_type_ids, attention_mask]
2        3        1       1 -0.994778    0         3  <그렇다> 감정이입이 잘 되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...  [input_ids, token_type_ids, attention_mask]
3        4        1       1 -0.994778    0         4  <중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다일의 변수가 생길 수 있고 변...  [input_ids, token_type_ids, attention_mask]
4        5        1       1 -0.994778    0         5  <아니다> 평정심을 유지 못하는 편입니다 머릿속은 백지화가 된 상태로 말도 제대로 ...  [input_ids, token_type_ids, attention_mask]


In [ ]:
from transformers import DataCollatorWithPadding

# define collator function when padding per batch is needed
#TODO: data_collator가 아닌 torch의 Packing 을 이용하는 것과 성능 비교가 필요함
data_collator = DataCollatorWithPadding(tokenizer=train_dataset.tokenizer) if env_dict['padding_per_batch'] else None

# Dataloader
train_dataloder = DataLoader(
    train_dataset,
    batch_size  = env_dict['batch_size'],
    shuffle     = env_dict['shuffle'],
    collate_fn  = data_collator
)

# Prepare model

In [ ]:
# Model
from transformers import BertConfig